# Preprocessing: BJH027


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [2]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [3]:
## Prep paths ##

subject = 'BJH027'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH027/ieeg/BJH027_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH027/ieeg/BJH027_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 2983799 =   1043.000 ...  1491.899 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH027/ieeg/BJH027_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH027/ieeg/BJH027_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 2983799 =   1043.000 ...  1491.899 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

EMPTY noref
EMPTY_2 noref
EMPTY_3 noref
EMPTY_4 noref
REF1 noref
REF2 noref
AL1 AL2
AL2 AL3
AL3 AL4
AL4 AL5
AL5 AL6
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL10
AL10 AL11
AL11 AL12
AL12 AL13
AL13 AL14
BL1 BL2
BL2 BL3
BL3 BL4
BL4 BL5
BL5 BL6
BL6 BL7
BL7 BL8
BL8 BL9
BL9 BL10
BL10 BL11
BL11 BL12
BL12 BL13
BL13 BL14
BL14 BL15
BL15 BL16
CL1 CL2
CL2 CL3
CL3 CL4
CL4 CL5
CL5 CL6
CL6 CL7
CL7 CL8
CL8 CL9
CL9 CL10
CL10 CL11
CL11 CL12
CL12 CL13
CL13 CL14
CL14 CL15
CL15 CL16
DL1 DL2
DL2 DL3
DL3 DL4
DL4 DL5
DL5 DL6
DL6 DL7
DL7 DL8
DL8 DL9
DL9 DL10
DL10 DL11
DL11 DL12
DL12 DL13
DL13 DL14
DL14 DL15
DL15 DL16
EL1 EL2
EL2 EL3
EL3 EL4
EL4 EL5
EL5 EL6
EL6 EL7
EL7 EL8
EL8 EL9
EL9 EL10
FL1 FL2
FL2 FL3
FL3 FL4
FL4 FL5
FL5 FL6
FL6 FL7
FL8 noref
GL1 GL3
GL2 noref
GL3 GL4
GL4 GL5
GL5 GL6
GL6 GL7
GL7 GL8
GL8 GL12
GL9 noref
GL10 noref
GL11 noref
GL12 GL13
GL13 GL14
HL1 noref
HL2 noref
HL3 noref
HL4 noref
HL5 HL7
HL6 noref
HL7 HL8
HL8 HL9
HL9 HL10
HL10 HL11
HL11 HL12
HL12 HL13
HL13 HL14
HL14 HL15
HL15 HL16
IL1 IL2
IL2 IL3
IL3

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2983799  =      0.000 ...  1491.899 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=181, n_times=2983800
    Range : 0 ... 2983799 =      0.000 ...  1491.899 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, BL14-BL15, BL15-BL16, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, CL12-CL13, CL13-CL14, CL14-CL15, CL15-CL16, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, DL12-DL13, DL13-DL14, DL14-DL15, DL15-DL16, EL1-EL2, EL2-EL3, EL3-EL4, EL4-EL5, EL5-EL6, EL6-EL7, EL7-EL8, EL8-EL9, EL9-EL10, FL1-FL2, FL2-FL3, FL3-FL4, FL4-FL5, FL5-FL6, FL6-FL7, GL1-GL3, GL3-GL4, 

/tmp/ipykernel_145017/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2983799  =      0.000 ...  1491.899 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=181, n_times=2983800
    Range : 0 ... 2983799 =      0.000 ...  1491.899 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, BL14-BL15, BL15-BL16, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, CL12-CL13, CL13-CL14, CL14-CL15, CL15-CL16, 

/tmp/ipykernel_145017/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_raw_clean_data.fif
[done]


In [10]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 160, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.


/tmp/ipykernel_145017/2669736035.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]


Channels marked as bad:
['C3', 'C4', 'CZ', 'EKG1', 'EKG2', 'EMPTY', 'EMPTY_2', 'EMPTY_221', 'EMPTY_222', 'EMPTY_223', 'EMPTY_224', 'EMPTY_225', 'EMPTY_226', 'EMPTY_227', 'EMPTY_228', 'EMPTY_229', 'EMPTY_230', 'EMPTY_231', 'EMPTY_3', 'EMPTY_4', 'F10', 'F3', 'F4', 'F7', 'F8', 'F9', 'FL8', 'FP1', 'FP2', 'FPZ', 'FZ', 'GL10', 'GL11', 'GL2', 'GL9', 'HL1', 'HL2', 'HL3', 'HL4', 'HL6', 'HR1', 'HR2', 'HR3', 'HR4', 'HR5', 'JL1', 'KL1', 'O1', 'O2', 'OZ', 'P3', 'P4', 'P7', 'P8', 'PZ', 'REF1', 'REF2', 'T7', 'T8', 'JL5-JL6', 'DL2-DL3', 'DL4-DL5']


In [15]:
# From interactive sessions
interactive_annot = bp_filtered_fif.annotations
for x in range(0, len(bp_filtered_fif.annotations)):
    print(interactive_annot[x])
    
    
interactive_annot.save('../../../raw_data/BJH027/ieeg/saved_annotations.fif', overwrite = True)    

OrderedDict([('onset', 33.747841), ('duration', 0.5715827941894531), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 53.189335), ('duration', 0.6636009216308594), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 75.686996), ('duration', 2.199005126953125), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 81.30397), ('duration', 0.9767913818359375), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 167.154037), ('duration', 3.8839569091796875), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 179.843002), ('duration', 2.6880035400390625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 209.884003), ('duration', 1.7610015869140625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 248.965286), ('duration', 0.0), ('description', 'BAD_'), ('orig_time', None)])
OrderedDict([('onset', 250.322281), ('duration', 1.147125244140625), ('description', 'BA

/tmp/ipykernel_145017/1026889880.py:7: RuntimeWarning: This filename (../../../raw_data/BJH027/ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  interactive_annot.save('../../../raw_data/BJH027/ieeg/saved_annotations.fif', overwrite = True)


## Epoching the data 

### Onset

In [16]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.


/tmp/ipykernel_145017/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...


/tmp/ipykernel_145017/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


### Trial End

In [18]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.


/tmp/ipykernel_145017/749539899.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts


/tmp/ipykernel_145017/749539899.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...
Loading data for 120 events and 20001 original time points ...


### Last Away

In [4]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,7,45300,0,1
1,1,6,56300,0,1
2,2,11,68400,0,1
3,3,5,83800,0,1
4,4,9,96200,0,1
...,...,...,...,...,...
202,234,14,2815300,0,1
203,235,8,2825700,0,1
204,236,18,2836100,0,1
205,237,19,2848000,0,1


In [5]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [6]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.
Not setting metadata
207 matching events found
No baseline correction applied


/tmp/ipykernel_76648/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts


/tmp/ipykernel_76648/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 104 events and 20001 original time points ...
Loading data for 103 events and 20001 original time points ...


### First Dot

In [23]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,7,41900,0,1
1,1,6,54300,0,1
2,2,11,66100,0,1
3,3,5,80200,0,1
4,4,9,94300,0,1
...,...,...,...,...,...
218,235,8,2823900,0,1
219,236,18,2833700,0,1
220,237,19,2845800,0,1
221,238,10,2857800,0,1


In [24]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [25]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 112 events and 18001 original time points ...


/tmp/ipykernel_145017/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_145017/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 111 events and 18001 original time points ...


## First Move

In [26]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,7,41000,0,1
1,1,6,54100,0,1
2,2,11,65300,0,1
3,3,5,79400,0,1
4,4,9,93600,0,1
...,...,...,...,...,...
235,235,8,2823200,0,1
236,236,18,2833000,0,1
237,237,19,2845000,0,1
238,238,10,2857000,0,1


In [27]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [28]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_145017/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_145017/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...


## Ghost Attack

In [30]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,4,9,95600,0,1
1,8,13,143600,0,1
2,11,12,171600,0,1
3,12,15,182700,0,1
4,13,14,194400,0,1
...,...,...,...,...,...
67,225,16,2703800,0,1
68,226,3,2727500,0,1
69,232,7,2795800,0,1
70,234,14,2815300,0,1


In [31]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [32]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 2983799 =   1113.000 ...  1491.899 secs
Ready.
Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 72 events and 20001 original time points ...


/tmp/ipykernel_145017/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_145017/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH027/ieeg/BJH027_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
